In [1]:
import pandas as pd
from geopy.distance import geodesic
import numpy as np

In [2]:
danios = pd.read_csv('/content/drive/MyDrive/2_Cursos/4_Henry/PF_Sismos/data/danios.csv', encoding= "ANSI", delimiter=";")
sismos = pd.read_csv('/content/drive/MyDrive/2_Cursos/4_Henry/PF_Sismos/data/sismos_con_estado.csv')

In [30]:
sismos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44945 entries, 0 to 44944
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id sismo     44945 non-null  int64  
 1   fecha        44945 non-null  object 
 2   hora         44945 non-null  object 
 3   latitud      44945 non-null  float64
 4   longitud     44945 non-null  float64
 5   profundidad  44945 non-null  float64
 6   magnitud     44945 non-null  float64
 7   pais         44945 non-null  object 
 8   estado       44945 non-null  object 
 9   id_lugar     44945 non-null  int64  
dtypes: float64(4), int64(2), object(4)
memory usage: 3.4+ MB


In [3]:
sismos = sismos[(sismos["pais"] == 'Perú') | (sismos["pais"] == 'Bolivia') | (sismos["pais"] == 'Brasil') |
               (sismos["pais"] == 'Ecuador') | (sismos["pais"] == 'Chile') | (sismos["pais"] == 'Colombia') |
               (sismos["pais"] == 'Argentina')]


In [4]:
sismos['fecha'] = pd.to_datetime(sismos['fecha'])
sismos = sismos[(sismos['fecha'].dt.year >= 2002) & (sismos['fecha'].dt.year <= 2018)]

In [33]:
danios.columns

Index(['Nº', 'COD_EMER', 'FECHA', 'AÑO', 'MESES', 'UBIGEO', 'DEPARTAMENTO',
       'PROVINCIA', 'DISTRITO', 'EMERGENCIAS', 'REGION NATURAL', 'FALLEC',
       'DESAPAR', 'HERIDOS', 'DAMNIF', 'AFECT', 'VIV DEST', 'VIV AFECT',
       'IIEE DEST', 'IIEE AFECT', 'CCSS DEST', 'CCSS AFECT', 'HAS PERD',
       'HAS AFECT'],
      dtype='object')

In [34]:
danios.head(1)

,Nº,COD_EMER,FECHA,AÑO,MESES,UBIGEO,DEPARTAMENTO,PROVINCIA,DISTRITO,EMERGENCIAS,...,DAMNIF,AFECT,VIV DEST,VIV AFECT,IIEE DEST,IIEE AFECT,CCSS DEST,CCSS AFECT,HAS PERD,HAS AFECT
0,1,NaN,1/1/2003,2003,1-ene,30208,APURÍMAC,ANDAHUAYLAS,PACOBAMBA,INUNDACIÓN,...,158,140,35,28,1,0,0,0,0,0


In [35]:
#Dejo las columnas que me interesan
df = danios[['FECHA',  'DEPARTAMENTO', 'PROVINCIA', 'DISTRITO', 'EMERGENCIAS','FALLEC', 'DESAPAR', 'HERIDOS', 'DAMNIF', 'AFECT', 'VIV DEST', 'VIV AFECT']]

In [36]:
#Filtro solo por sismo
df = df[df["EMERGENCIAS"] == "SISMO"]

In [37]:
df['FECHA'] = pd.to_datetime(df['FECHA'])

C:\Users\Clarita\AppData\Local\Temp\ipykernel_8376\2142312632.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['FECHA'] = pd.to_datetime(df['FECHA'])


In [38]:
df.sort_values('FECHA', inplace=True)

In [39]:
df.reset_index(drop=True, inplace=True)

In [40]:
df.rename(columns={"DEPARTAMENTO": "estado"}, inplace= True)

In [41]:
df["estado"].replace({'AMAZONAS':  'Amazonas',
                      'APURIMAC' : 'Apurímac' , 
                      'APURÍMAC': 'Apurímac',
                      'AREQUIPA' :  'Arequipa',
                      'AYACUCHO' :  'Ayacucho',
                      'CAJAMARCA' :  'Cajamarca',
                      'CALLAO' : "Callao",
                      'CUSCO' : "Cusco",
                      'HUANCAVELICA' : 'Huancavelica' ,
                      'HUÁNUCO': 'Huánuco',
                      'ICA': "Ica",
                      'JUNIN' : "Junín",
                      'JUNÍN'  : "Junín",
                      'LA LIBERTAD' :"La Libertad",
                      'LAMBAYEQUE' : "Lambayeque",
                      'LIMA': "Lima", 
                      'LORETO' : "Loreto",
                      'MOQUEGUA' : "Moquegua",
                      'PASCO':  'Pasco',
                      'PIURA' :  'Piura',
                      'PUNO' :  'Puno', 
                      'SAN MARTÍN' : "San Martin",
                      'TACNA': "Tacna",
                      'TUMBES': "Tumbes",
                      'UCAYALI': "Ucayali",
                      'ÁNCASH': "Ancash"}, inplace=True)

In [42]:
df = df.groupby(['FECHA', 'estado']).agg({
    'FALLEC': 'sum',
    'DESAPAR': 'sum',
    'HERIDOS': 'sum',
    'DAMNIF': 'sum',
    'AFECT': 'sum',
    'VIV DEST': 'sum',
    'VIV AFECT': 'sum'
}).reset_index()

In [43]:
df['afectados']= df[['AFECT', 'DAMNIF']].max(axis=1)

In [44]:
df.drop(columns=['AFECT', 'DAMNIF'],inplace=True )

In [45]:
df.rename(columns={'FECHA': "fecha",'FALLEC': "fallecidos", 'DESAPAR': "desaparecidos", 'HERIDOS': "heridos", 'VIV DEST': "viv_destr",
       'VIV AFECT': "viv_afect"}, inplace= True)

In [46]:
df.insert(0, 'id_danio', df.index+1)

In [47]:
df.columns

Index(['id_danio', 'fecha', 'estado', 'fallecidos', 'desaparecidos', 'heridos',
       'viv_destr', 'viv_afect', 'afectados'],
      dtype='object')

In [48]:
df = df.reindex(columns=['id_danio', 'fecha', 'estado', 'afectados','fallecidos', 'desaparecidos', 'heridos',
       'viv_destr', 'viv_afect' ])

In [49]:
df.insert(3, 'pais', 'Perú')


In [50]:
df

,id_danio,fecha,estado,pais,afectados,fallecidos,desaparecidos,heridos,viv_destr,viv_afect
0,1,2003-08-08,Apurímac,Perú,295,0,0,0,11,268
1,2,2003-08-08,Cusco,Perú,4498,0,0,0,239,905
2,3,2003-08-13,Cusco,Perú,60,0,0,0,10,0
3,4,2003-08-24,Ica,Perú,7,2,0,0,2,0
4,5,2003-08-26,Arequipa,Perú,0,0,0,7,0,0
...,...,...,...,...,...,...,...,...,...,...
337,338,2017-10-10,Moquegua,Perú,0,0,0,0,0,0
338,339,2017-10-10,Tacna,Perú,0,0,0,0,0,0
339,340,2017-10-12,Puno,Perú,0,0,0,0,0,0
340,341,2017-10-14,Lima,Perú,0,0,0,0,0,2


In [51]:
#sismos.ciudad = sismos.ciudad.apply(lambda x: x.lower().replace("á","a").replace("é","e").replace("í","i").replace("ó","o").replace("ú","u").title())

In [52]:
#df.ciudad = df.ciudad.apply(lambda x: x.lower().replace("á","a").replace("é","e").replace("í","i").replace("ó","o").replace("ú","u").title())

In [55]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [56]:
geolocator = Nominatim(user_agent='my_app')

In [57]:
df

,id_danio,fecha,estado,pais,afectados,fallecidos,desaparecidos,heridos,viv_destr,viv_afect
0,1,2003-08-08,Apurímac,Perú,295,0,0,0,11,268
1,2,2003-08-08,Cusco,Perú,4498,0,0,0,239,905
2,3,2003-08-13,Cusco,Perú,60,0,0,0,10,0
3,4,2003-08-24,Ica,Perú,7,2,0,0,2,0
4,5,2003-08-26,Arequipa,Perú,0,0,0,7,0,0
...,...,...,...,...,...,...,...,...,...,...
337,338,2017-10-10,Moquegua,Perú,0,0,0,0,0,0
338,339,2017-10-10,Tacna,Perú,0,0,0,0,0,0
339,340,2017-10-12,Puno,Perú,0,0,0,0,0,0
340,341,2017-10-14,Lima,Perú,0,0,0,0,0,2


In [58]:
# Función para obtener coordenadas
def obtener_coordenadas(lugar):
    location = geolocator.geocode(lugar)
    if location is not None:
        return location.latitude, location.longitude
    else:
        return None, None



# Obtener las coordenadas y agregarlas al DataFrame
df[['latitud', 'longitud']] = df[['estado', 'pais']].apply(lambda row: obtener_coordenadas(row['estado'] + ', ' + row['pais']), axis=1, result_type='expand')

# Imprimir el DataFrame resultante
print(df)


     id_danio      fecha    estado  pais  afectados  fallecidos  \
0           1 2003-08-08  Apurímac  Perú        295           0   
1           2 2003-08-08     Cusco  Perú       4498           0   
2           3 2003-08-13     Cusco  Perú         60           0   
3           4 2003-08-24       Ica  Perú          7           2   
4           5 2003-08-26  Arequipa  Perú          0           0   
..        ...        ...       ...   ...        ...         ...   
337       338 2017-10-10  Moquegua  Perú          0           0   
338       339 2017-10-10     Tacna  Perú          0           0   
339       340 2017-10-12      Puno  Perú          0           0   
340       341 2017-10-14      Lima  Perú          0           0   
341       342 2017-11-08  Arequipa  Perú          0           0   

     desaparecidos  heridos  viv_destr  viv_afect    latitud   longitud  
0                0        0         11        268 -14.000000 -73.000000  
1                0        0        239        9

In [59]:
# Coordenadas de los puntos
punto1 = (df.loc[0,"latitud"], df.loc[0,"longitud"])
punto2 = (sismos.loc[9908,"latitud"], sismos.loc[9908,"longitud"])

# Calcular la distancia entre los puntos
distancia = geodesic(punto1, punto2).kilometers

# Imprimir la distancia
print(f"La distancia entre los puntos es: {distancia} kilómetros")


La distancia entre los puntos es: 1496.4292625256646 kilómetros


In [6]:
df

,id_danio,fecha,estado,pais,afectados,fallecidos,desaparecidos,heridos,viv_destr,viv_afect,latitud,longitud
0,1,2003-08-08,Apurímac,Perú,295,0,0,0,11,268,-14.000000,-73.000000
1,2,2003-08-08,Cusco,Perú,4498,0,0,0,239,905,-13.517089,-71.978536
2,3,2003-08-13,Cusco,Perú,60,0,0,0,10,0,-13.517089,-71.978536
3,4,2003-08-24,Ica,Perú,7,2,0,0,2,0,-14.332500,-75.499722
4,5,2003-08-26,Arequipa,Perú,0,0,0,7,0,0,-16.000000,-72.250000
...,...,...,...,...,...,...,...,...,...,...,...,...
337,338,2017-10-10,Moquegua,Perú,0,0,0,0,0,0,-16.833333,-70.916667
338,339,2017-10-10,Tacna,Perú,0,0,0,0,0,0,-17.666667,-70.333333
339,340,2017-10-12,Puno,Perú,0,0,0,0,0,0,-15.000000,-70.000000
340,341,2017-10-14,Lima,Perú,0,0,0,0,0,2,-12.062107,-77.036526


In [81]:
df.to_csv("daño_con_coordenadas.csv", index= False)

In [5]:
df= pd.read_csv("../estefi/daño_con_coordenadas.csv")

In [7]:
df_merged = df.copy()

In [8]:
# Crear columna de distancia mínima y id_sismo en md_merged
df_merged['distancia_corta'] = np.nan
df_merged['id_sismo'] = np.nan

# Iterar sobre cada fila en md_merged
for idx, row in df_merged.iterrows():
    fecha = row['fecha']
    coordenadas_df = (row['latitud'], row['longitud'])

    # Filtrar sismos por la misma fecha
    sismos_fecha = sismos[sismos['fecha'] == fecha]
    max_magnitud = sismos_fecha["magnitud"].max()

    # Inicializar variables para almacenar la distancia mínima y el id_sismo correspondiente
    distancia_minima = float('inf')
    id_sismo_minimo = None

    # Iterar sobre cada fila en sismos_fecha
    for _, sismo_row in sismos_fecha.iterrows():
        coordenadas_sismo = (sismo_row['latitud'], sismo_row['longitud'])
        distancia = geodesic(coordenadas_df, coordenadas_sismo).kilometers

        # Actualizar la distancia mínima y el id_sismo si se encuentra una distancia más corta
        if distancia < distancia_minima and max_magnitud == sismo_row["magnitud"]:
            distancia_minima = distancia
            id_sismo_minimo = sismo_row['id sismo']

    # Actualizar las columnas en md_merged
    df_merged.at[idx, 'distancia_corta'] = distancia_minima
    df_merged.at[idx, 'id_sismo'] = id_sismo_minimo

In [11]:
df_merged.isnull().sum()

id_danio           0
fecha              0
estado             0
pais               0
afectados          0
fallecidos         0
desaparecidos      0
heridos            0
viv_destr          0
viv_afect          0
latitud            0
longitud           0
distancia_corta    0
id_sismo           0
dtype: int64

In [10]:
df_merged.dropna(subset=['id_sismo'], inplace= True)


In [14]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 280 entries, 0 to 341
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_danio         280 non-null    int64  
 1   fecha            280 non-null    object 
 2   estado           280 non-null    object 
 3   pais             280 non-null    object 
 4   afectados        280 non-null    int64  
 5   fallecidos       280 non-null    int64  
 6   desaparecidos    280 non-null    int64  
 7   heridos          280 non-null    int64  
 8   viv_destr        280 non-null    int64  
 9   viv_afect        280 non-null    int64  
 10  latitud          280 non-null    float64
 11  longitud         280 non-null    float64
 12  distancia_corta  280 non-null    float64
 13  id_sismo         280 non-null    int32  
dtypes: float64(3), int32(1), int64(7), object(3)
memory usage: 31.7+ KB


In [15]:
#df_merged['id_sismo'] = df_merged['id_sismo'].replace(np.inf, None).replace(np.nan, None)
#df_merged = df_merged[df_merged['id_sismo'] != pd.NA]
df_merged['id_sismo'] = df_merged['id_sismo'].astype(int)
df_merged["distancia_corta"] = round(df_merged["distancia_corta"],2)

In [30]:
df_merged.columns

Index(['id_danio', 'fecha', 'estado', 'pais', 'afectados', 'fallecidos',
       'desaparecidos', 'heridos', 'viv_destr', 'viv_afect', 'latitud',
       'longitud', 'distancia_corta', 'id_sismo'],
      dtype='object')

In [31]:
df_merged.drop(columns= ['latitud','longitud','distancia_corta'], inplace=True)

In [32]:
df_merged.to_csv("daños_final.csv", index= False)